In [7]:
import pandas as pd
import sys

sys.path.append("..")

from src.auth.aurora import get_portal_client

In [8]:
portal_client = get_portal_client()

In [9]:
site_id = 8928

In [10]:

tests_df = portal_client.query_to_pandas(
    """SELECT
        test.id,
        test.created_at,
        site.template_id,
        site.protocol_id,
        mode_change.toc_id,
        test.panel,
        test.domain,
        test.loop,
        test.address,
        test.sub_address
    FROM topaz.api_service_tests AS test
    INNER JOIN topaz.api_mode_changes AS mode_change
    ON test.mode_change_id = mode_change.id
    INNER JOIN topaz.api_sites AS site
    ON test.site_id = site.id
    WHERE test.site_id = %(site_id)s
    AND test.created_at >= '2024-01-01 00:00:00'""",
    {"site_id": site_id}
)
missing_task_records = False
changed_asset_type_records = []
tests_df['panel'] = tests_df['panel'].fillna(-999)
tests_df['domain'] = tests_df['domain'].fillna(-999)
tests_df['loop'] = tests_df['loop'].fillna(-999)
tests_df['address'] = tests_df['address'].fillna(-999)
tests_df['sub_address'] = tests_df['sub_address'].fillna(-999)
tests_df.loc[tests_df['sub_address'] == '', 'sub_address'] = -999
tests_df.loc[tests_df['sub_address'] == 0, 'sub_address'] = -999

for (template_id, protocol_id, toc_id), test_df in tests_df.groupby(["template_id", "protocol_id", "toc_id"]):
    template_df = portal_client.query_to_pandas(
        """SELECT
            id AS task_id,
            asset_type,
            created_at,
            deleted_at
        FROM topaz.api_service_tasks
        WHERE template_id = %(template_id)s
        AND `type` IN (1,3)""",
        {"template_id": int(template_id)}
    )
    template_df["asset_type"] = template_df["asset_type"].str.lower()
    toc_df = portal_client.query_to_pandas(
        """SELECT `panel`, `domain`, `loop`, `address`, `sub_address`, `type`, `lcs_device_type`, `created_at` AS toc_created_at, `deleted_at` AS toc_deleted_at
        FROM topaz.api_toc_rows
        WHERE toc_id = %(toc_id)s""",
        {"toc_id": int(toc_id)}
    )
    toc_df["type"] = toc_df["type"].str.lower()
    toc_df["lcs_device_type"] = toc_df["lcs_device_type"].str.lower()
    toc_df['panel'] = toc_df['panel'].fillna(-999)
    toc_df['domain'] = toc_df['domain'].fillna(-999)
    toc_df['loop'] = toc_df['loop'].fillna(-999)
    toc_df['address'] = toc_df['address'].fillna(-999)
    toc_df['sub_address'] = toc_df['sub_address'].fillna(-999)
    toc_df.loc[toc_df['sub_address'] == 0, 'sub_address'] = -999
    toc_df["legacy_device_type"] = toc_df["lcs_device_type"].copy()
    other_mask = toc_df["lcs_device_type"] == "other"
    toc_df.loc[other_mask, "lcs_device_type"] = toc_df.loc[other_mask, "type"]
    merged_df = pd.merge(
        how="inner",
        left=test_df,
        left_on=["panel", "domain", "loop", "address", "sub_address"],
        right=toc_df,
        right_on=["panel", "domain", "loop", "address", "sub_address"],
    )
    time_mask = (merged_df["created_at"] >= merged_df["toc_created_at"]) & (
        (
            merged_df["created_at"] < merged_df["toc_deleted_at"]
        ) | merged_df["toc_deleted_at"].isnull()
    )
    merged_df = merged_df[time_mask]

    template_df["key"] = 1
    blank_asset_mask = template_df["asset_type"] == ""
    all_assets = pd.DataFrame(
        {"asset_type": toc_df["lcs_device_type"].unique()}
    )
    all_assets["key"] = 1
    full_template_df = pd.concat([
        pd.merge(
            left=template_df.loc[blank_asset_mask, ["key", "task_id", "created_at", "deleted_at"]],
            right=all_assets.loc[:, ["key", "asset_type"]],
            on="key"
        ),
        template_df.loc[~blank_asset_mask, ["key", "task_id", "created_at", "deleted_at", "asset_type"]]
    ])
    normal_merged_df = pd.merge(
        how="inner",
        left=merged_df,
        left_on="lcs_device_type",
        right=full_template_df,
        right_on="asset_type",
    )

    if protocol_id == 34:
        merged_df["el_key"] = 1
    else:
        merged_df["el_key"] = 0
    template_df["el_key"] = 1 * (template_df["asset_type"] == "el")

    el_merged_df = pd.merge(
        how="inner",
        left=merged_df[merged_df["el_key"] == 1],
        left_on="el_key",
        right=template_df[template_df["el_key"] == 1],
        right_on="el_key",
    )

    if protocol_id == 35:
        merged_df["ext_key"] = 1 * (
            (merged_df["legacy_device_type"] != "other")
        )
    else:
        merged_df["ext_key"] = 0
    template_df["ext_key"] = 1 * (template_df["asset_type"] == "ext")

    ext_merged_df = pd.merge(
        how="inner",
        left=merged_df[merged_df["ext_key"] == 1],
        left_on="ext_key",
        right=template_df[template_df["ext_key"] == 1],
        right_on="ext_key",
    )

    merged_df = pd.concat([
        normal_merged_df,
        el_merged_df,
        ext_merged_df
    ])

    timing_mask = (merged_df["created_at_x"] >= merged_df["created_at_y"]) & (
        (merged_df["created_at_x"] < merged_df["deleted_at"]) | (
            merged_df["deleted_at"].isnull()
        )
    )
    merged_df = merged_df[timing_mask]
    if merged_df.empty:
        print("----> No tasks")
        continue
    for (test_id,), tasks_df in merged_df.groupby(["id"]):
        task_record_df = portal_client.query_to_pandas(
            """SELECT tr.test_id, tr.task_id, t.asset_type
            FROM topaz.api_service_tasks_record AS tr
            LEFT JOIN topaz.api_service_tasks AS t
            ON tr.task_id = t.id
            WHERE test_id = %(test_id)s""",
            {"test_id": int(test_id)}
        )
        additional_assets = set(task_record_df["asset_type"]) - set(tasks_df["asset_type"]) - {''}
        if len(additional_assets) > 0:
            changed_asset_type_records.append(test_id)
        joined_tasks = pd.merge(
            how="left",
            left=tasks_df,
            left_on=["id", "task_id"],
            right=task_record_df,
            right_on=["test_id", "task_id"]
        )
        missing_tasks = joined_tasks[joined_tasks["asset_type_y"].isnull()]
        if not missing_tasks.empty:
            missing_task_records = True
            break
    if missing_task_records:
        break
assert not (missing_task_records)
if missing_task_records:
    missing_tasks_string = ", ".join(str(int(task)) for task in missing_tasks["task_id"].values)
    print(f"{test_id} missing tasks {missing_tasks_string}")
if (len(changed_asset_type_records) > 0):
    additional_assets_string = ", ".join(str(int(test)) for test in changed_asset_type_records)
    print(f"Additional assets on tests: {additional_assets_string}")

Additional assets on tests: 11377682, 11377748
